# Setup
-  Follow the setup instructions based on your preferred environment!

## Local

One of our key goals in designing this assignment is to allow you to complete most of the preliminary implementation work locally.  
We highly recommend that you **pass all tests locally** using the provided `hw4_data_subset` before moving to a GPU runtime.  
To do this, simply:

### Create a new conda environment
```bash
# Be sure to deactivate any active environments first
conda create -n hw4 python=3.12.4
```

### Activate the conda environment
```bash
conda activate hw4
```

### Install the dependencies using the provided `requirements.txt`
```bash
pip install --no-cache-dir --ignore-installed -r requirements.txt
```

### Ensure that your notebook is in the same working directory as the `Handout`
This can be achieved by:
1. Physically moving the notebook into the handout directory.
2. Changing the notebook’s current working directory to the handout directory using the os.chdir() function.

### Open the notebook and select the newly created environment from the kernel selector.

If everything was done correctly, You should see atleast the following files in your current working directory after running `!ls`:
```
.
├── README.md
├── requirements.txt
├── hw4lib/
├── mytorch/
├── tests/
└── hw4_data_subset/
```

## Colab

### Step 1: Get your handout
- See writeup for recommended approaches.

In [1]:
# Example: My preferred approach
import os
# Settings -> Developer Settings -> Personal Access Tokens -> Token (classic)
os.environ['GITHUB_TOKEN'] = "your_token_here"

GITHUB_USERNAME = "your_username_here"
REPO_NAME       = "your_repo_name_here"
TOKEN = os.environ.get("GITHUB_TOKEN")
repo_url        = f"https://{TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

Cloning into 'your_repo_name_here'...
fatal: could not read Password for 'https://your_token_here@github.com': No such device or address


In [2]:
# To pull latest changes (Must be in the repo dir, use pwd/ls to verify)
!cd {REPO_NAME} && git pull

/bin/bash: line 1: cd: your_repo_name_here: No such file or directory


### Step 2: Install Dependencies
- `NOTE`: Your runtime will be restarted to ensure all dependencies are updated.
- `NOTE`: You will see a runtime crashed message, this was intentionally done. Simply move on to the next cell.

In [ ]:
%pip install --no-deps -r IDL-HW4/requirements.txt
import os
os.kill(os.getpid(), 9) # NOTE: This will restart the your colab Python runtime (required)!

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'IDL-HW4/requirements.txt'


### Step 3: Obtain Data

- `NOTE`: This process will automatically download and unzip data for both `HW4P1` and `HW4P2`.  


In [ ]:
!curl -L -o /content/f25-hw4-data.zip https://www.kaggle.com/datasets/cmu11785/f25-11785-hw4-data
!unzip -q -o /content/f25-hw4-data.zip -d /content/hw4_data
!rm -rf /content/f25-hw4-data.zip
!du -h --max-depth=2 /content/hw4_data

### Step 4: Move to Handout Directory
You must be within the handout directory for the library imports to work!

- `NOTE`: You may have to repeat running this command anytime you restart your runtime.
- `NOTE`: You can do a `pwd` to check if you are in the right directory.
- `NOTE`: The way it is setup currently, Your data directory should be one level up from your project directory. Keep this in mind when you are setting your `root` in the config file.

If everything was done correctly, You should see atleast the following files in your current working directory after running `!ls`:
```
.
├── README.md
├── requirements.txt
├── hw4lib/
├── mytorch/
├── tests/
└── hw4_data_subset/

```

In [ ]:
import os
os.chdir('IDL-HW4')
!ls

# Imports

- If your setup was done correctly, you should be able to run the following cell without any issues.

In [ ]:
from hw4lib.data import (
    H4Tokenizer,
    LMDataset,
    verify_dataloader
)
from hw4lib.model import (
    CausalMask,
    PadMask,
    PositionalEncoding,
    DecoderOnlyTransformer
)
from hw4lib.utils import (
    create_optimizer,
    create_scheduler,
    plot_lr_schedule
)
from hw4lib.trainers import (
    LMTrainer,
)
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import yaml
import gc
import torch
from torchinfo import summary
import os
import json
import tarfile
import shutil
import wandb
import yaml
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


# Implementations

- `NOTE`: All of these implementations have detailed specification, implementation details, and hints in their respective source files. Make sure to read all of them in their entirety to understand the implementation details!

## MyTorch Implementations
- Modify your `Linear` implementation from HW1P1 to support arbitrary number of dimensions in `mytorch/nn/linear.py`.
- Modify your `Softmax` implementation from HW1P1 to support arbitrary number of dimensions in `mytorch/nn/activation.py`.
- Implement the `ScaledDotProductAttention` class in `mytorch/nn/scaled_dot_product_attention.py`.
- Implement the `MultiHeadAttention` class in `mytorch/nn/multi_head_attention.py`.
- Run the cell below to check your implementations.


In [ ]:
!python -m tests.test_mytorch


Running tests for category: Linear
--------------------------------------------------------------------------------

[01/01]    Running:  Linear Tests
Testing Linear Layer ...
Test Passed: Linear Forward
Test Passed: Linear Backward
[01/01]    PASSED:   Linear Tests


Running tests for category: Softmax
--------------------------------------------------------------------------------

[01/01]    Running:  Softmax Tests
Testing Softmax ...
Test Passed: Softmax Forward
Test Passed: Softmax Backward
[01/01]    PASSED:   Softmax Tests


Running tests for category: ScaledDotProductAttention
--------------------------------------------------------------------------------

[01/01]    Running:  ScaledDotProductAttention Tests
Testing Scaled Dot Product Attention ...
Test Passed: Scaled Dot Product Attention Forward
Test Passed: Scaled Dot Product Attention Backward
[01/01]    PASSED:   ScaledDotProductAttention Tests


Running tests for category: MultiHeadAttention
----------------------------

## Dataset Implementation
- Familiarize yourself with the `tokenize`, `encode`, and `decode` methods of the `H4Tokenizer` class in `hw4lib/data/tokenizer.py`. You will need to make use of these methods in both `HW4P1` and `HW4P2` both in the dataset implementations and during decoding.
- Implement the `LMDataset` class in `hw4lib/data/lm_dataset.py`.
    - You will have to implement parts of `__init__` and completely implement the `__len__`, `__getitem__` and `collate_fn` methods.
- Run the cell below to check your implementation.


In [ ]:
!python -m tests.test_dataset_lm

Loading transcripts for train partition...

Running tests for category: LMDataset
--------------------------------------------------------------------------------

[01/01]    Running:  Test the LMDataset class
Testing __init__ method ...
Test Passed: Text files are sorted.
Test Passed: Shifted and golden transcripts are aligned.
Test Passed: All transcripts are decoded correctly after removing SOS and EOS tokens.
Testing __getitem__ method ...
Test Passed: SOS and EOS tokens are correctly placed for samples.
Test Passed: All transcripts are decoded correctly after removing SOS and EOS tokens.
Testing collate_fn method ...
Test Passed: Transcript batch has correct dimensions (2D tensor).
Test Passed: Transcript lengths are consistent with the batch size.
Test Passed: All sequences are padded to the same length.
Test Passed: Padding values are correct.
Test Passed: Batch transcripts are of correct data type.
[01/01]    PASSED:   Test the LMDataset class


                                


100%|██████████| 267/267 [00:01<00:00, 195.35it/s]


## Model Implementations
#### Overview:
- Implement the `CausalMask` and `PadMask` functions in `hw4lib/modules/masks.py` to handle masking.
- Implement the `PositionalEncoding` class in `hw4lib/model/positional_encoding.py` to handle positional encoding.
- Implement the Transformer Sublayers: `SelfAttentionLayer` and `FeedForwardLayer` classes in `hw4lib/model/sublayers.py`.
- Implement the Transformer Layer: `SelfAttentionDecoderLayer` class in `hw4lib/model/decoder_layers.py`.
- Implement the `DecoderOnlyTransformer` class in `hw4lib/model/transformers.py`.
- Run the cells below to check your implementation.
- `NOTE`: Besides the `DecoderOnlyTransformer` (P1 mandatory, P2 optional), you will use all of the above implementations in both `HW4P1` and `HW4P2`!

### Masks
- Implement the `PadMask` and `CausalMask` functions in `hw4lib/modules/masks.py`.
- Run the cell below to check your implementation.
- You will need to make use of these masks in both `HW4P1` and `HW4P2`.

#### Causal Mask

In [ ]:
!python -m tests.test_mask_causal


Running tests for category: CausalMask
--------------------------------------------------------------------------------

[01/01]    Running:  Test the causal mask generation
Testing Causal Mask ...
Test Passed: Causal mask generation is correct.
[01/01]    PASSED:   Test the causal mask generation


                                  Test Summary                                  
Category:    CausalMask                    
Results:     1/1 tests passed (100.0%)


#### Padding Mask

In [ ]:
!python -m tests.test_mask_padding


Running tests for category: PaddingMask
--------------------------------------------------------------------------------

[01/01]    Running:  Test the padding mask generation
Testing Padding Mask ...
Test Passed: Padding mask generation is correct.
[01/01]    PASSED:   Test the padding mask generation


                                  Test Summary                                  
Category:    PaddingMask                   
Results:     1/1 tests passed (100.0%)


#### Optional: Visualize your Masks

In [ ]:
# Dummy data
_d_model   = 64
_x         = torch.zeros(4, 20, _d_model)
_x_len     = torch.tensor([5, 15, 10, 20])
_x_causal  = CausalMask(_x)
_x_padding = PadMask(_x, _x_len)

# Create figure with two subplots side by side
fig, mask_axs = plt.subplots(1, 2, figsize=(12, 4))

# Plot masks
masks_and_titles = [
    (_x_padding, "Padding Mask"),
    (_x_causal, "Causal Mask")
]

# Plot each mask
images = []
for i, (mask, title) in enumerate(masks_and_titles):
    im = mask_axs[i].imshow(mask, cmap="gray", aspect='auto')
    mask_axs[i].set_title(title, fontsize=8)
    images.append(im)

# Add colorbar at the bottom
fig.subplots_adjust(bottom=0.2)  # Make space for colorbar
cbar_ax = fig.add_axes([0.15, 0.1, 0.7, 0.02])  # [left, bottom, width, height]
cbar = plt.colorbar(images[0], cax=cbar_ax, orientation='horizontal')
cbar.ax.set_xlabel('Mask Values', labelpad=5, fontsize=8)
cbar.set_ticks([0, 1])
cbar.set_ticklabels(['Attend (0)', 'Ignore/Mask (1)'])
cbar.ax.tick_params(labelsize=6)

plt.show()

NameError: name 'torch' is not defined

### Positional Encoding
- Implement the `PositionalEncoding` class in `hw4lib/model/positional_encoding.py`.
- Run the cell below to check your implementation.
- You will need to make use of this positional encoding in both `HW4P1` and `HW4P2`.

In [ ]:
!python -m tests.test_positional_encoding


Running tests for category: PositionalEncoding
--------------------------------------------------------------------------------

[01/01]    Running:  Test the positional encoding generation
Testing Positional Encoding Shape ...
Test Passed: Positional Encoding Shape is Correct
Testing Positional Encoding Values ...
Test Passed: Positional Encoding Values are Correct
Testing Positional Encoding Forward ...
Test Passed: Positional Encoding Forward is Correct
[01/01]    PASSED:   Test the positional encoding generation


                                  Test Summary                                  
Category:    PositionalEncoding            
Results:     1/1 tests passed (100.0%)


#### Optional: Visualize your Positional Encoding

In [ ]:
# Create sample positional encoding
d_model = 64
max_len = 100
pos_encoding = PositionalEncoding(d_model=d_model, max_len=max_len)
pe = pos_encoding.pe.squeeze(0).numpy()  # Remove batch dimension and convert to numpy

# Create figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot 1: Positional encoding matrix
im = ax1.imshow(pe, aspect='auto', cmap='RdBu',
                extent=[0, d_model, max_len, 0])  # Flip y-axis to show position top-to-bottom
plt.colorbar(im, ax=ax1, label='Encoding Value')
ax1.set_xlabel('Dimension')
ax1.set_ylabel('Position')
ax1.set_title('Positional Encoding Matrix')
ax1.grid(False)

# Plot 2: Sinusoidal patterns
dimensions = [0, 15, 31, 47, 63]  # Plot first few dimensions
for dim in dimensions:
    ax2.plot(pe[:, dim], label=f'dim {dim}')
ax2.set_xlabel('Position')
ax2.set_ylabel('Encoding Value')
ax2.set_title('Sinusoidal Patterns for Different Dimensions')
ax2.legend()
ax2.grid(True)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

### Transformer Sublayers
- Implement the Transformer Sublayers: `SelfAttentionLayer`, and `FeedForwardLayer` classes in `hw4lib/model/sublayers.py`.
- Run the cell below to check your implementation.
- You will need to make use of all of these sublayers in both `HW4P1` and `HW4P2`.

In [ ]:
!python -m tests.test_sublayer_selfattention


Running tests for category: SelfAttentionLayer
--------------------------------------------------------------------------------

[01/01]    Running:  Test the self-attention sublayer
Testing initialization ...
Test Passed: All layers exist and are instantiated correctly
Testing forward shapes ...
Test Passed: Forward pass returns the correct shapes
Testing padding mask behaviour ...
Testing padding mask behaviour ...
Test Passed: Padding mask is applied correctly
Testing self-attention mask behaviour ...
Test Passed: Self-attention mask is applied correctly
Testing self-attention residual ...
Test Passed: Residual connection is applied correctly
[01/01]    PASSED:   Test the self-attention sublayer


                                  Test Summary                                  
Category:    SelfAttentionLayer            
Results:     1/1 tests passed (100.0%)


In [ ]:
!python -m tests.test_sublayer_feedforward


Running tests for category: FeedForwardLayer
--------------------------------------------------------------------------------

[01/01]    Running:  Test the feedforward sublayer
Testing initialization ...
Test Passed: All layers exist and match reference implementation
Testing forward shapes ...
Test Passed: Forward pass returns correct shapes for various input dimensions
Testing feed-forward network behavior ...
Test Passed: Feed-forward network transforms input appropriately
Testing residual connection ...
Test Passed: Residual connection is working correctly
Testing layer normalization ...
Test Passed: Layer normalization is being applied correctly
Testing forward pass order ...
Test Passed: Forward pass operations are in correct order
[01/01]    PASSED:   Test the feedforward sublayer


                                  Test Summary                                  
Category:    FeedForwardLayer              
Results:     1/1 tests passed (100.0%)


### Transformer Self-Attention Decoder Layer
- Implement the Transformer Layer: `SelfAttentionDecoderLayer` class in `hw4lib/model/decoder_layers.py`.
- Run the cell below to check your implementation.
- You will need to make use of this sublayer in `HW4P2`.

In [ ]:
!python -m tests.test_decoderlayer_selfattention


Running tests for category: SelfAttentionDecoderLayer
--------------------------------------------------------------------------------

[01/01]    Running:  Test the self-attention decoder layer
Testing initialization ...
Test Passed: All sublayers exist and are initialized correctly
Testing forward shapes ...
Test Passed: Forward shapes are as expected
Testing sublayer interaction ...
Test Passed: Sublayers interact correctly
[01/01]    PASSED:   Test the self-attention decoder layer


                                  Test Summary                                  
Category:    SelfAttentionDecoderLayer     
Results:     1/1 tests passed (100.0%)


### Decoder-Only Transformer

- Implement the `DecoderOnlyTransformer` class in `hw4lib/model/transformers.py`.
- Run the cell below to check your implementation.
- You will need to make use of in `HW4P1` and optionally `HW4P2`.

In [ ]:
!python -m tests.test_transformer_decoder_only


Running tests for category: DecoderOnlyTransformer
--------------------------------------------------------------------------------

[01/01]    Running:  Test the decoder only transformer
Testing initialization...
Test Passed: All components initialized correctly
Testing forward pass...
Test Passed: Forward pass works correctly
Testing forward propagation order...
Test Passed: Forward propagation order is correct
[01/01]    PASSED:   Test the decoder only transformer


                                  Test Summary                                  
Category:    DecoderOnlyTransformer        
Results:     1/1 tests passed (100.0%)


## Decoding Implementation
- Implement the `generate_greedy` method of the `SequenceGenerator` class in `hw4lib/decoding/sequence_generator.py`.
- Run the cell below to check your implementation.

In [1]:
!python -m tests.test_decoding --mode greedy


Running tests for category: Decoding
--------------------------------------------------------------------------------

[01/01]    Running:  Test greedy decoding
Testing Single Batch Greedy Search ...
[01/01]    ERROR:    Test greedy decoding
           Error:    'list' object has no attribute 'to'

           Traceback: Traceback (most recent call last):
  File "c:\Users\ADMIN\Desktop\11685\11685homework\IDL-HW4\IDL-HW4\tests\testing_framework.py", line 153, in __run_tests_category
    test["func"]()
  File "c:\Users\ADMIN\Desktop\11685\11685homework\IDL-HW4\IDL-HW4\tests\test_decoding.py", line 354, in <lambda>
    'func': lambda: test_greedy_decoding(SequenceGenerator, tokenizer),
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ADMIN\Desktop\11685\11685homework\IDL-HW4\IDL-HW4\tests\test_decoding.py", line 298, in test_greedy_decoding
    test_greedy_search_single_batch(generator, tokenizer)
  File "c:\Users\ADMIN\Desktop\11685\11685homework\I

## Trainer Implementation
You will have to do some minor in-filling for the `LMTrainer` class in `hw4lib/trainers/lm_trainer.py` before you can use it.
- Fill in the `TODO`s in the `__init__`.
- Fill in the `TODO`s in the `_train_epoch`.
- Fill in the `TODO`s in the `_validate_epoch`.
- Fill in the `TODO`s in the `generate` method.
- Fill in the `TODO`s in the `train` method.

`WARNING`: There are no test's for this. Implement carefully!

# Experiments
From this point onwards you may want to switch to a `GPU` runtime.
- `OBJECTIVE`: You must achieve a per-character perplexity ≤ 3.5 in order to get points for Task 2.

## Config
- You can use the `config.yaml` file to set your config for your ablation study.

---
### Notes:

- Set `tokenization: token_type:` to specify your desired tokenization strategy
- You will need to set the root path to your `hw4p1_data` folder in `data: root:`. This will depend on your setup. For eg. if you are following out setup instruction:
  - `PSC`: `"/ocean/projects/cis250019p/mzhang23/TA/HW4/hw4p1_data"`
  - `Colab:`: `"/content/hw4_data/hw4p1_data"`
  - `Kaggle:`: `"/kaggle/input/s25-hw4-data/hw4p1_data"`
- There's extra configurations in the `optimizer` section which will only be relevant if you decide to use the `create_optimizer` function we've provided in `hw4lib/utils/create_optimizer.py`.
- `BE CAREFUL` while setting numeric values. Eg. `1e-4` will get serialized to a `str` while `1.0e-4` gets serialized to float.


In [ ]:
%%writefile config.yaml

Name                      : "Enter-Name-Here"

###### Tokenization ------------------------------------------------------------
tokenization:
  token_type                : "char"       # [char, 1k, 5k, 10k]
  token_map :
      'char': 'hw4lib/data/tokenizer_jsons/tokenizer_char.json'
      '1k'  : 'hw4lib/data/tokenizer_jsons/tokenizer_1000.json'
      '5k'  : 'hw4lib/data/tokenizer_jsons/tokenizer_5000.json'
      '10k' : 'hw4lib/data/tokenizer_jsons/tokenizer_10000.json'

###### Dataset -----------------------------------------------------------------
data:                    # Currently setup for Colab assuming out setup
  root                 : "hw4_data/hw4p1_data"  # TODO: Set the root path of your data
  train_partition      : "train"  # train
  val_partition        : "val"    # val
  test_partition       : "test"   # test
  subset               : 1.0      # Load a subset of the data (for debugging, testing, etc
  batch_size           : 256      #
  NUM_WORKERS          : 2        # Set to 0 for CPU

###### Network Specs -------------------------------------------------------------
model: # Decoder-Only Language Model (HW4P1)
  d_model                   : 256
  d_ff                      : 1024
  num_layers                : 2
  num_heads                 : 2
  dropout                   : 0.0
  layer_drop_rate           : 0.0
  weight_tying              : False

###### Common Training Parameters ------------------------------------------------
training:
  use_wandb                   : True   # Toggle wandb logging
  wandb_run_id                : "none" # "none" or "run_id"
  resume                      : False  # Resume an existing run (run_id != 'none')
  epochs                      : 20
  gradient_accumulation_steps : 1
  wandb_project               : "Set-Project-Name-Here" # wandb project to log to

###### Loss ----------------------------------------------------------------------
loss: # Just good ol' CrossEntropy
  label_smoothing: 0.0

###### Optimizer -----------------------------------------------------------------
optimizer:
  name: "adam" # Options: sgd, adam, adamw
  lr: 5.0e-4   # Base learning rate

  # Common parameters
  weight_decay: 0.0001

  # Parameter groups
  param_groups:
    - name: self_attn
      patterns: []  # Will match all parameters containing keywords set their learning rate to 0.0001
      lr: 0.0001    # LR for self_attn
      layer_decay:
        enabled: False
        decay_rate: 0.8

    - name: ffn
      patterns: [] # Will match all parameters containing "ffn" and set their learning rate to 0.0001
      lr: 0.0001   # LR for ffn
      layer_decay:
        enabled: False
        decay_rate: 0.8

  # Layer-wise learning rates
  layer_decay:
    enabled: False
    decay_rate: 0.75

  # SGD specific parameters
  sgd:
    momentum: 0.9
    nesterov: True
    dampening: 0

  # Adam specific parameters
  adam:
    betas: [0.9, 0.999]
    eps: 1.0e-8
    amsgrad: False

  # AdamW specific parameters
  adamw:
    betas: [0.9, 0.999]
    eps: 1.0e-8
    amsgrad: False

###### Scheduler -----------------------------------------------------------------
scheduler:
  name: "cosine"  # Options: reduce_lr, cosine, cosine_warm

  # ReduceLROnPlateau specific parameters
  reduce_lr:
    mode: "min"  # Options: min, max
    factor: 0.1  # Factor to reduce learning rate by
    patience: 10  # Number of epochs with no improvement after which LR will be reduced
    threshold: 0.0001  # Threshold for measuring the new optimum
    threshold_mode: "rel"  # Options: rel, abs
    cooldown: 0  # Number of epochs to wait before resuming normal operation
    min_lr: 0.0000001  # Minimum learning rate
    eps: 1.0e-8  # Minimal decay applied to lr

  # CosineAnnealingLR specific parameters
  cosine:
    T_max: 15  # Maximum number of iterations
    eta_min: 1.0e-8  # Minimum learning rate
    last_epoch: -1

  # CosineAnnealingWarmRestarts specific parameters
  cosine_warm:
    T_0: 4  # Number of iterations for the first restart
    T_mult: 4  # Factor increasing T_i after each restart
    eta_min: 0.0000001  # Minimum learning rate
    last_epoch: -1

  # Warmup parameters (can be used with any scheduler)
  warmup:
    enabled: True
    type: "exponential"  # Options: linear, exponential
    epochs: 5
    start_factor: 0.1
    end_factor: 1.0

In [ ]:
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

## Tokenizer

In [ ]:
Tokenizer = H4Tokenizer(
    token_map  = config['tokenization']['token_map'],
    token_type = config['tokenization']['token_type']
)

NameError: name 'H4Tokenizer' is not defined

## Datasets

In [ ]:
train_dataset  = LMDataset(
    partition  = config['data']['train_partition'],
    config     = config['data'],
    tokenizer  = Tokenizer
)

val_dataset    = LMDataset(
    partition  = config['data']['val_partition'],
    config     = config['data'],
    tokenizer  = Tokenizer
)

test_dataset   = LMDataset(
    partition  = config['data']['test_partition'],
    config     = config['data'],
    tokenizer  = Tokenizer
)

gc.collect()

## Dataloaders

In [ ]:
train_loader    = DataLoader(
    dataset     = train_dataset,
    batch_size  = config['data']['batch_size'],
    shuffle     = True,
    num_workers = config['data']['NUM_WORKERS'] if device == 'cuda' else 0,
    pin_memory  = True,
    collate_fn  = train_dataset.collate_fn
)

val_loader      = DataLoader(
    dataset     = val_dataset,
    batch_size  = config['data']['batch_size'],
    shuffle     = False,
    num_workers = config['data']['NUM_WORKERS'] if device == 'cuda' else 0,
    pin_memory  = True,
    collate_fn  = val_dataset.collate_fn
)

test_loader     = DataLoader(
    dataset     = test_dataset,
    batch_size  = config['data']['batch_size'],
    shuffle     = False,
    num_workers = config['data']['NUM_WORKERS'] if device == 'cuda' else 0,
    pin_memory  = True,
    collate_fn  = test_dataset.collate_fn
)

### Dataloader Verification

In [ ]:
verify_dataloader(train_loader)

NameError: name 'verify_dataloader' is not defined

In [ ]:
verify_dataloader(val_loader)

In [ ]:
verify_dataloader(test_loader)

## Calculate Max Transcript Length




Calculating the maximum transcript length across your dataset is a crucial step when working with certain transformer models.
-  We'll use sinusoidal positional encodings that must be precomputed up to a fixed maximum length.
- This maximum length is a hyperparameter that determines:
  - How long of a sequence your model can process
  - The size of your positional encoding matrix
  - Memory requirements during training and inference
- `Requirements`: For this assignment, ensure your positional encodings can accommodate at least the longest sequence in your dataset to prevent truncation. However, you can set this value higher if you anticipate using your language model to work with longer sequences in future tasks (hint: this might be useful for P2! 😉).

In [ ]:
max_transcript_length = max(train_dataset.text_max_len, val_dataset.text_max_len, test_dataset.text_max_len)
print("="*50)
print(f"{'Global Max Transcript Length':<30} : {max_transcript_length}")
print("="*50)

## Model

In [ ]:
model_config = config['model']
model_config.update({
    'max_len': max_transcript_length,
    'num_classes': Tokenizer.vocab_size
})
model = DecoderOnlyTransformer(**model_config)

# Get some inputs from the text loader
for batch in train_loader:
    shifted_transcripts, golden_transcripts, transcript_lengths = batch
    print("Shape of shifted_transcripts : ", shifted_transcripts.shape)
    print("Shape of golden_transcripts  : ", golden_transcripts.shape)
    print("Shape of transcript_lengths  : ", transcript_lengths.shape)
    break

model_stats = summary(model, input_data=[shifted_transcripts, transcript_lengths])
print(model_stats)

## Wandb

In [ ]:
wandb.login(key="your_wandb_api_key_here")

## Trainer

Every time you run the trainer, it will create a new directory in the `expts` folder with the following structure:
```
expts/
    └── {run_name}/
        ├── config.yaml
        ├── model_arch.txt
        ├── checkpoints/
        │   ├── checkpoint-best-metric-model.pth
        │   └── checkpoint-last-epoch-model.pth
        ├── attn/
        │   └── {attention visualizations}
        └── text/
            └── {generated text outputs}
```

In [ ]:
trainer = LMTrainer(
    model=model,
    tokenizer=Tokenizer,
    config=config,
    run_name="test-lm",
    config_file="config.yaml",
    device=device
)

### Setup Optimizer and Scheduler

You can set your own optimizer and scheduler by setting the class members in the `LMTrainer` class.
Eg:
```python
trainer.optimizer = optim.AdamW(model.parameters(), lr=config['optimizer']['lr'], weight_decay=config['optimizer']['weight_decay'])
trainer.scheduler = optim.lr_scheduler.CosineAnnealingLR(trainer.optimizer, T_max=config['training']['epochs'])
```

We also provide a utility function to create your own optimizer and scheduler with the congig and some extra bells and whistles. You are free to use it or not. Do read their code and documentation to understand how it works (`hw4lib/utils/*`).


#### Setting up the optimizer

In [ ]:
trainer.optimizer = create_optimizer(
    model=model,
    opt_config=config['optimizer']
)

#### Creating a test scheduler and plotting the learning rate schedule

In [ ]:
test_scheduler = create_scheduler(
    optimizer=trainer.optimizer,
    scheduler_config=config['scheduler'],
    train_loader=train_loader,
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps']
)

plot_lr_schedule(
    scheduler=test_scheduler,
    num_epochs=config['training']['epochs'],
    train_loader=train_loader,
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps']
)

#### Setting up the scheduler

In [ ]:
trainer.scheduler = create_scheduler(
    optimizer=trainer.optimizer,
    scheduler_config=config['scheduler'],
    train_loader=train_loader,
    gradient_accumulation_steps=config['training']['gradient_accumulation_steps']
)

# Train
- Set your epochs

In [ ]:
trainer.train(train_loader, val_loader, epochs=config['training']['epochs'])

# Evaluate


In [ ]:
test_metrics, test_generation_results = trainer.evaluate(test_loader)
# Cleanup
trainer.cleanup()

# Submission
To submit your assignment, you will need to create a `handin.tar` with the following directory structure:

```
handin/
├── mytorch/                     # Your implemented modules
├── test_metrics.json            # Results from evaluation
├── test_generated_results.json  # Sample text generations
└── model_arch.txt               # Model architecture summary
```

- Simply run the cell below once you are satisfied with your current state and this will create the `handin.tar` file.
- After running the above cell, you should see the handin.tar file in the current directory
- Upload the `handin.tar` file to the `HW4P1` assignment on Autolab.

In [ ]:
# Create temporary handin directory
if os.path.exists('handin'):
    shutil.rmtree('handin')
os.makedirs('handin')

# Copy mytorch directory
shutil.copytree('mytorch', 'handin/mytorch')

# Save final results
with open('handin/test_metrics.json', 'w') as f:
    json.dump(test_metrics, f, indent=4)

with open('handin/test_generated_results.json', 'w') as f:
    json.dump(test_generation_results['greedy'], f, indent=4)

# Save model architecture
with open('handin/model_arch.txt', 'w') as f:
    f.write(str(model_stats))

# Create tar file with all exclusions handled by filter
with tarfile.open('handin.tar', 'w') as tar:
    def filter_files(tarinfo):
        # Skip unwanted files
        if any(pattern in tarinfo.name for pattern in [
            '.DS_Store',
            '__pycache__',
            '.pyc'
        ]):
            return None
        return tarinfo

    tar.add('handin', arcname='handin', filter=filter_files)

# Cleanup
shutil.rmtree('handin')

print("Created handin.tar successfully!")

## After running the above cell, you should see the handin.tar file in the current directory
!ls